In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time
import multiprocessing as mp
import os
from itertools import repeat

import LGTp as lgt

In [2]:
os.cpu_count()

8

In [3]:
_e = np.identity(4,dtype=int)
coordinates = np.arange(4)

def calcPlaq(config,x,y,z,t,mu,nu):
    
    conf_shape = np.array(np.shape(config)[:-1])
    n = np.array([x,y,z,t])
    
    plaq = (config[tuple(n)+(mu,)]
            *config[tuple((n+_e[mu])%conf_shape)+(nu,)]
            *(config[tuple((n+_e[nu])%conf_shape)+(mu,)].conj())
            *(config[tuple(n)+(nu,)].conj())
           )
    
    return plaq

def calcPlaqSum(config):
    
    N = len(config)
    conf_shape = np.array(np.shape(config)[:-1])
    sp = 0.
    
    for n in np.ndindex(tuple(conf_shape)):
        for nu in range(4):
            for mu in range(nu):
                sp += calcPlaq(config,n[0],n[1],n[2],n[3],mu,nu).real
    
    return sp/N**4/(4.-1.)/2.

def calcPoly(config):
    
    conf_shape = np.array(np.shape(config)[:-1])
    p = 0.
    
    for n in np.ndindex(tuple(conf_shape[:-1])):
        p_local = 1.
        for t in range(conf_shape[3]):
            p_local *= config[tuple(n)+(t,)+(3,)]
        p += p_local
    
    return p/(4.**3)

def calcPolyAbs(config):

    conf_shape = np.array(np.shape(config)[:-1])
    p = 0.
    
    for n in np.ndindex(tuple(conf_shape[:-1])):
        p_local = 1.
        for t in range(conf_shape[3]):
            p_local *= config[tuple(n)+(t,)+(3,)]
        p += p_local
    
    return np.sqrt((p/(4.**3))*(p/(4.**3)).conj())

# def calcTopCharg(config):
    

In [4]:
def detect_equi_t(O_cold, O_hot, tol = 1e-2, step = 20):
    
    t = len(O_cold)
    
    for i in range(len(O_cold)-step):
        
        diff = np.abs(np.average(np.abs(O_cold[i:i+step]) - np.abs(O_hot[i:i+step])))
        
        if diff < tol:
            t = i
            return t
    
    return t

In [5]:
def detect_ac_t(ac, tol = 1e-2, step = 3):
    
    t = len(ac)
    
    for i in range(len(ac) - step):
        
        diff = np.average(np.abs(ac[i:i+step]))
        
        if diff < tol:
            t = i
            return t
    
    return t

In [6]:
def autocorrelation(conf, O, t):
    
    N = len(conf) - t
    
#     cor_hist = np.empty(N)
    
    o1o2 = 0.
    o1 = 0.
    o2 = 0.
    
    o1o1 = 0.
    o2o2 = 0.
    
    for i in range(N):
        
        o1o2 += O(conf[i])*O(conf[i+t])/N
        
        o1 += O(conf[i])/N
        o2 += O(conf[i+t])/N
        
        o1o1 += O(conf[i])*O(conf[i])/N
        o2o2 += O(conf[i+t])*O(conf[i+t])/N
        
    o1_err = np.sqrt(o1o1 - o1*o1)
    o2_err = np.sqrt(o2o2 - o2*o2)
        
    cor_t = o1o2 - o1*o2
        
    return cor_t/o1_err/o2_err

In [7]:
# Equilibrating phase detection
def calc_teq(bare_arg, O, tol=1e-2, step=100,seed=0):
    t_eq = step

    O_cold = []
    O_hot = []

    u1_cold = lgt.Lattice([N,N,N,N])
    u1_cold.init_fields('U1','Cold',seed)

    u1_hot = lgt.Lattice([N,N,N,N])
    u1_hot.init_fields('U1','Hot',seed)
    
    for i in range(step):
        lgt.metropolis(u1_cold,bare_arg)
        lgt.metropolis(u1_hot,bare_arg)

        diff = np.abs(O(u1_cold.field) - O(u1_hot.field))
        O_cold.append(O(u1_cold.field))
        O_hot.append(O(u1_hot.field))

        if diff < tol:
            t_eq = i
            break
    
    return t_eq

In [8]:
def calc_tac(bare_arg, t_eq, tol=1e-2, steps = 50,seed=0):
    
    u1_ac = lgt.Lattice([N,N,N,N])
    u1_ac.init_fields('U1','Cold',seed)

    n_equi_ac = t_eq
    for i in range(n_equi_ac):
        lgt.metropolis(u1_ac, bare_arg)

    n_conf_ac = steps
    conf_ac = []
    Plaq_ac = []
    for e in range(n_conf_ac):
        lgt.metropolis(u1_ac,bare_arg)
        conf_ac.append(u1_ac.field)
        Plaq = calcPlaqSum(u1_ac.field)
        Plaq_ac.append(Plaq)
        
    ac_hist = np.empty(len(conf_ac))

    for i in range(len(conf_ac)):
        ac_hist[i] = autocorrelation(conf_ac,calcPlaqSum,i)

    t_ac = detect_ac_t(ac_hist) + 1
    
    return t_ac

In [9]:
# calculate equilibrating phase
beta_list = np.linspace(1.0,1.02,12)

nt = len(beta_list)

ensem = []
N = 6
n_conf = 200
# n_conf = 2

In [10]:
# for b in range(nt):
def simulate(b):
#     start = time.time()
    seed = int(beta_list[b]*1000)

    bare_parameters = {'beta':beta_list[b]}

    t_eq = calc_teq(bare_parameters,calcPlaqSum,seed=seed)

    t_ac = calc_tac(bare_parameters, t_eq, seed=seed)
    
    u1 = lgt.Lattice([N,N,N,N])
    u1.init_fields('U1','Cold',seed)
    
    for i in range(t_eq):
#         mcmove(config,beta)
        lgt.metropolis(u1,bare_parameters)
    
    conf = []
        
    for e in range(n_conf*t_ac):
#         mcmove(config,beta)
        lgt.metropolis(u1,bare_parameters)
        
        if not e%t_ac:
            conf.append(u1.field)
    
#     conf_name = './data/U1%d/U1_b%0.3fN%d.npy' %(N,beta,N)
#     np.save(conf_name, ensemble)

In [11]:
# start = time.time()
# simulate(0)
# due = time.time() - start
# print("time span:",due)

In [ ]:
start = time.time()

p = mp.Pool(3)
res = p.map(simulate, range(nt))
p.close()
p.join()

due = time.time() - start
print("time span:",due)